In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.3 MB/s eta 0:00:00


#test model

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import os

adapter_path = "llama-3.1-8b-custom-adapter"
base_model_name = "meta-llama/Llama-3.2-3B-Instruct" # IMPORTANT: Use the 8B model you trained on
my_token = os.environ.get("")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading base model: {base_model_name}...")
# NOTE: Add your Hugging Face token if required
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    token=my_token
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    token=my_token,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

print(f"Loading adapter from: {adapter_path}...")
model = PeftModel.from_pretrained(base_model, adapter_path)

model.eval()
print("Model and adapter loaded successfully!")

Loading base model: meta-llama/Llama-3.2-3B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter from: /content/drive/MyDrive/llama-3.1-8b-custom-adapter...
Model and adapter loaded successfully!


In [2]:
# Example:
test_df = pd.read_csv('/content/test_dataset.csv')
test_df.head()

,date,likes,content,username,inferred company,media_url,media_type,generated_caption
0,2020-02-27 18:09:10,357,"13' GOAL! After a lengthy VAR review, Lucas Al...",bayer04_en,bayer,https://pbs.twimg.com/media/ERzSRc3WkAAib0o?fo...,photo,The image features a soccer player in a red je...
1,2018-07-31 15:55:23,171,Go to gifs somewhere on your phone and type in...,boyerweathers,cameron,https://pbs.twimg.com/tweet_video_thumb/DjcgAX...,video,The image depicts a scene inside what appears ...
2,2020-06-16 19:24:46,21,*sweet* story alert ☺️ A customer’s hope for q...,WalmartWorld,walmart,https://pbs.twimg.com/media/EaqCkKqXkAAhny5?fo...,photo,The image shows a rectangular cake with a clea...
3,2019-08-14 11:28:24,122,Amazing morning at Ballydoyle <mention> watchi...,olibellracing,bell,https://pbs.twimg.com/media/EB7VW_yWwAIrioO?fo...,photo,The image depicts a stable or training facilit...
4,2020-01-03 17:44:27,304,NOW: Climate activists are marching in front o...,greenpeaceusa,greenpeace,https://pbs.twimg.com/media/ENX9ROnWoAA-tCB?fo...,photo,The image depicts a large gathering of people ...


In [3]:
RESULTS_CSV_PATH = "/content/drive/MyDrive/test_results_with_generation.csv"

In [4]:
def make_prompt(row):
    """
    Build a text-to-text training prompt for Task-2 (content simulation).
    (Copied exactly from your training script)
    """
    date = str(row.get("date", ""))
    likes = str(row.get("likes", ""))
    username = str(row.get("username", ""))
    media_type = str(row.get("media_type", ""))
    company = str(row.get("inferred company", row.get("inferred_company", "")))
    caption = str(row.get("generated_caption", ""))

    prompt = f"""
You are a content-generation model trained to simulate Twitter marketing posts.

Your task:
Given structured metadata input, generate realistic marketing tweet text. wuth brand indentity, usename, comapny
and only give tweet not other things.

Input:
<date> {date}
<likes> {likes}
<username> {username}
<company> {company}
<media> {media_type}
<media caption> {caption}

Output (content):
"""
    return "\n".join(line.strip() for line in prompt.strip().split("\n"))

def create_inference_prompt(row):
    """
    Llama 3 chat template ke hisaab se final inference prompt banata hai.
    """
    user_prompt_content = make_prompt(row)

    messages = [
        {"role": "user", "content": user_prompt_content}
    ]

    # apply_chat_template special tokens add kar dega
    # add_generation_prompt=True se ye "assistant" ka part shuru kar dega
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # <-- Inference ke liye True hona zaroori hai
    )

    return formatted_prompt

In [5]:
TEST_CSV_PATH="/content/test_dataset.csv"

In [6]:
import pandas as pd
import torch
from tqdm import tqdm


print(f"Loading test data from: {TEST_CSV_PATH}")
try:
    df_test = pd.read_csv(TEST_CSV_PATH)
    df_test=df_test.iloc[1:500]
except FileNotFoundError:
    print(f"Error: Test file not found at {TEST_CSV_PATH}")

BATCH_SIZE = 50
results = []
print(f"Starting inference loop with batch size {BATCH_SIZE}...")

for i in tqdm(range(0, len(df_test), BATCH_SIZE)):

    batch_df = df_test.iloc[i:i + BATCH_SIZE]

    prompts = [create_inference_prompt(row) for _, row in batch_df.iterrows()]

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    ).to("cuda")

    input_ids = inputs.input_ids

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=[
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ]
        )

    generated_ids = outputs[:, input_ids.shape[1]:]

    generated_content_batch = tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )

    for content in generated_content_batch:
        results.append(content.strip())

    # Optional: Batch ka pehla result debug karne ke liye
    # print(f"--- PROMPT (Batch {i//BATCH_SIZE + 1} - Item 1) ---")
    # print(prompts[0])
    # print(f"--- GENERATED (Batch {i//BATCH_SIZE + 1} - Item 1) ---")
    # print(generated_content_batch[0].strip())
    # print("-----------------\n")


print("Inference complete.")

if len(results) == len(df_test):
    df_test['generated_content'] = results
    df_test.to_csv(RESULTS_CSV_PATH, index=False)
    print(f"Successfully saved results to: {RESULTS_CSV_PATH}")
    print(df_test.head())
else:
    print(f"Error: Mismatch in lengths. Expected {len(df_test)} results, but got {len(results)}.")

Loading test data from: /content/test_dataset.csv
Starting inference loop with batch size 60...


100%|██████████| 9/9 [24:36<00:00, 164.01s/it]

Inference complete.
Successfully saved results to: /content/drive/MyDrive/test_results_with_generation.csv
                  date  likes  \
1  2018-07-31 15:55:23    171   
2  2020-06-16 19:24:46     21   
3  2019-08-14 11:28:24    122   
4  2020-01-03 17:44:27    304   
5  2019-05-07 11:00:00      1   

                                             content       username  \
1  Go to gifs somewhere on your phone and type in...  boyerweathers   
2  *sweet* story alert ☺️ A customer’s hope for q...   WalmartWorld   
3  Amazing morning at Ballydoyle <mention> watchi...  olibellracing   
4  NOW: Climate activists are marching in front o...  greenpeaceusa   
5  With the latest extension to the Brexit proces...        TheAAie   

  inferred company                                          media_url  \
1          cameron  https://pbs.twimg.com/tweet_video_thumb/DjcgAX...   
2          walmart  https://pbs.twimg.com/media/EaqCkKqXkAAhny5?fo...   
3             bell  https://pbs.twimg.com/media/